In [6]:
import sys
import os
import pandas as pd
import numpy as np
import sqlite3
import time

from datetime import datetime, timedelta
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from itertools import combinations

In [7]:
driver = webdriver.Chrome()

driver.get('https://www.booking.com/index.ko.html?aid=318615&label=Korean_South_Korea-KO-KR-131246354044-lqHqiW6tNbRkHjsFQdmmhwS637818828162%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atidsa-1227182654382%3Alp1009842%3Ali%3Adec%3Adm%3Aag131246354044%3Acmp400536625&sid=1a2cdd5c26bf10a9c3ed2baaa7785c56&')

In [9]:

def rentcar_crawl(s_date, e_date):
    '''
    s_date, e_date: datetime\n
    s_time, e_time: str('HH:MM')
    '''
    driver.get('https://www.booking.com/index.ko.html?aid=318615&label=Korean_South_Korea-KO-KR-131246354044-lqHqiW6tNbRkHjsFQdmmhwS637818828162%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atidsa-1227182654382%3Alp1009842%3Ali%3Adec%3Adm%3Aag131246354044%3Acmp400536625&sid=1a2cdd5c26bf10a9c3ed2baaa7785c56&')


    # sleep
    time.sleep(5)

    # 날짜 선택할 selector 클릭
    date_select_button = driver.find_element(By.CLASS_NAME, 'd47738b911 e246f833f7 fe211c0731')
    date_select_button.click()
    time.sleep(0.5)

    # 시작일
    date_1 = driver.find_element(By.XPATH, f'//*[@data-date="{날짜}"]')
    date_1.click()
    time.sleep(0.5)

    # 종료일
    date_2 = driver.find_element(By.XPATH, f'//*[@data-date="{날짜}"]')
    date_2.click()
    time.sleep(0.5)
    
    driver.find_element(By.CLASS_NAME,"fc63351294 a822bdf511 d4b6b7a9e7 cfb238afa1 c938084447 f4605622ad aa11d0d5cd").click()
    time.sleep(0.5)
    

    # html parse
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # SK렌트카에서 구분지은 블록들
    hotels = soup.find_all(class_='d20f4628d0')

    for wrap in hotels:
            rentcar_sql(wrap, s_date_obj, e_date_obj)

def get_date_combinations(start_date, days):
    'start_date부터 days일 후까지 가능한 시작/종료일 조합'
    date_list = []
    for i in range(days):
        current_date = start_date + timedelta(days=i)
        date_list.append(current_date)
    
    combinations_list = list(combinations(date_list, 2))
    return combinations_list


def hotel_sql(wrap, s_date_obj: datetime, e_date_obj: datetime) :
    'sqlite3에 저장'
    conn = sqlite3.connect('hotel.db')
    cursor = conn.cursor()